# Define Graph Bound / Convergence tool for graphs

In [1]:
from scipy.sparse.linalg import eigsh
import networkx as nx
import numpy as np

def largest_eigen(A, num_iters=None, tol=None):
    if num_iters is None:
        num_iters = float('inf')  # By default only stop once tolerance criteria is met
    if tol is None:
        tol = 1e-12

    n = A.shape[0]  # Size of the matrix
    vec = np.ones(n)  # Starting with a constant vector

    iter_count = 0  # Initialize iteration counter
    while True:  # Loop until num_iters or convergence
        vec_next = A @ vec + vec # Add vec to ensure convergence ()
        norm = np.linalg.norm(vec_next)
        vec_diff = vec_next / norm - vec
        vec = vec_next / norm

        # Check for convergence
        if np.linalg.norm(vec_diff) < tol:
            break

        # Warn if exceeding 10,000 iterations
        if iter_count == 10000:
            print("Iteration count exceeded 10,000. Consider increasing tolerance or checking the matrix for convergence properties.")
        if iter_count == 100000:
            print("Iteration count exceeded 100,000. Consider increasing tolerance or checking the matrix for convergence properties.")
        if iter_count == 1000000:
            print("Iteration count exceeded 1,000,000. Consider increasing tolerance or checking the matrix for convergence properties.")

        if iter_count >= num_iters:
            print("Number of iterations exceeded the maximum (num_iters) allowed iterations.")
            break


        iter_count += 1

    eigenvalue = (vec.T @ (A @ vec)) / (vec.T @ vec)  # Rayleigh quotient for eigenvalue

    if not np.all(vec > 0):
        raise ValueError("The adjusted eigenvector does not have all positive entries.")

    return np.abs(eigenvalue), vec

def calculate_katz(A, alpha = 0.1, beta = 1.0, num_iters=10000, tol=None):
    if num_iters is None:
        num_iters = float('inf')  # By default only stop once tolerance criteria is met
    if tol is None:
        tol = 1e-12

    n = A.shape[0]  # Size of the matrix
    vec = np.zeros(n)  # Starting with a constant vector

    iter_count = 0  # Initialize iteration counter
    while iter_count < num_iters:  # Loop until num_iters or convergence
        vec_next = alpha * A @ vec + beta

        # Check for convergence
        if np.linalg.norm(vec_next - vec) < tol:
            print(f"Katz converged after {iter_count} iterations.")
            break

        vec = vec_next

        # Warn if exceeding 10,000 iterations
        if iter_count == 10000:
            print("Iteration count exceeded 10,000. Consider increasing tolerance or checking the matrix for convergence properties.")
        if iter_count == 100000:
            print("Iteration count exceeded 100,000. Consider increasing tolerance or checking the matrix for convergence properties.")
        if iter_count == 1000000:
            print("Iteration count exceeded 1,000,000. Consider increasing tolerance or checking the matrix for convergence properties.")

        if iter_count >= num_iters:
            print("Number of iterations exceeded the maximum (num_iters) allowed iterations.")
            break

        iter_count += 1

    return vec

def eigen_gap(A):
    val, _ = eigsh(A, k=2, which='LA')
    return val[1] - val[0]

def k_biggest_ev(A, k=1):
    val, _ = eigsh(A, k=k, which='LA')
    return val

def get_delta(A, lam, x):
    return A @ x - lam * x

def calculate_kmeans_cluster_loss(x, cluster_labels, centroids, mode=2):
    assert mode == 2
    total_loss = 0
    for i, color in enumerate(cluster_labels):
        centroid = centroids[color]
        distance = (x[i] - centroid) ** 2
        total_loss += distance
    return np.sqrt(total_loss)

In [2]:
import numpy as np
import random
from scipy.sparse import dok_matrix, csr_matrix, issparse

def create_partition_matrix(colors):
    unique_colors = np.unique(colors)
    n = len(colors)
    m = len(unique_colors)
    H = np.zeros((n, m), dtype=int)
    color_index = {color: idx for idx, color in enumerate(unique_colors)}

    for j in range(n):
        H[j, color_index[colors[j]]] = 1

    return H, color_index

def calculate_colored_degree_matrix(A, H):
    return A @ H

def create_color_node_mapping(colors):
    color_to_nodes = {}
    for idx, color in enumerate(colors):
        if color not in color_to_nodes:
            color_to_nodes[color] = set()
        color_to_nodes[color].add(idx)
    return color_to_nodes

def sample_new_adjacency_matrix(N, color_to_nodes, color_index, with_self_loops = False):
    n = N.shape[0]
    A_prime = dok_matrix((n, n), dtype=int) # Initialize the new adjacency matrix

    for c, nodes in color_to_nodes.items():
        if len(nodes) == 0:
            continue

        for i in range(n):
            # Determine the number of nodes to sample based on N[i][c] (colored degree)
            num_neighbors = int(N[i, color_index[c]])
            if num_neighbors > 0:
                # Sample nodes, ensuring no self-connections and respecting the number of available nodes
                possible_nodes = list(nodes) if with_self_loops else list(nodes - {i})
                sampled_neighbors = random.sample(possible_nodes, num_neighbors)
                for neighbor in sampled_neighbors:
                    A_prime[i, neighbor] = 1

    return csr_matrix(A_prime)

def nest_preserve_colored_out_degree(A, colors, seed = None, with_self_loops = False):
    if seed:
        np.random.seed(seed)
        random.seed(seed)

    if not with_self_loops:
        if issparse(A):
            diag_elements = A.diagonal()
        else:
            diag_elements = np.diag(A)
        assert np.all(diag_elements  == 0), "The adjacency matrix A should not contain any self-loops."
    H, color_index = create_partition_matrix(colors)
    N = calculate_colored_degree_matrix(A, H)
    color_to_nodes = create_color_node_mapping(colors)
    A_prime = sample_new_adjacency_matrix(N, color_to_nodes, color_index, with_self_loops=with_self_loops)
    return A_prime


# Example usage
colors = np.array([1, 2, 2, 3, 3, 3])
A = np.array([
    [0, 0, 0, 0, 0, 0],
    [0, 0, 1, 1, 0, 0],
    [1, 0, 0, 1, 1, 0],
    [1, 1, 0, 0, 1, 1],
    [1, 1, 1, 1, 0, 1],
    [1, 1, 1, 1, 1, 0]
])

A_prime = nest_preserve_colored_out_degree(A, colors, with_self_loops=False)

print("New Adjacency Matrix A':")
print(A_prime.toarray())


New Adjacency Matrix A':
[[0 0 0 0 0 0]
 [0 0 1 0 0 1]
 [1 0 0 0 1 1]
 [1 1 0 0 1 1]
 [1 1 1 1 0 1]
 [1 1 1 1 1 0]]


In [3]:
from anonymigraph.anonymization._external.nest_model._rewire import _rewire
import optimal1dclustering
import math

from anonymigraph.anonymization._external.nest_model.fast_wl import WL_fast

def calc_min_cluster_size(arr):
    arr = np.array(arr)
    unique, counts = np.unique(arr, return_counts=True)
    min_size = counts.min()
    return min_size


def get_convergence_data_of_nest(G, min_cluster_size = 1, alpha=0.1, beta = 1,  printing = False, max_k = float("Inf"), max_iter=None, tol=None, random_seed=44, with_self_loops=False):
    A_G = nx.adjacency_matrix(G).astype(np.float64)  # Get the adjacency matrix as a sparse matrix
    lam_G, _ = largest_eigen(A_G, max_iter, tol)
    degree_vector = np.array([G.degree[i] for i in G.nodes])

    _, max_degree = max(G.degree(), key=lambda pair: pair[1])

    print("Spectral Radius:", lam_G)
    print("max_degree:", max_degree)

    assert alpha < 1/lam_G, f"for katz to converge alpha needs to be smaller than 1/lam_G ({1/lam_G})"

    katz_cent = calculate_katz(A_G, alpha = alpha, beta=beta, num_iters=max_iter, tol=tol)
    katz_cent_norm = np.linalg.norm(katz_cent)

    data_dict = {}
    for k in range(1, min(G.number_of_nodes()//min_cluster_size, max_k)):
        print(f"{k}: ")
        results = {}
        mode = 2
        colors, centroids = optimal1dclustering.cluster(
            katz_cent, k, mode=mode, min_cluster_size=min_cluster_size
        )
        colors = np.array(colors)
        clusterLoss = calculate_kmeans_cluster_loss(katz_cent, colors, centroids, mode = mode)



        # Get graph spectrum statistics
        A_Ga = nest_preserve_colored_out_degree(A_G, colors, seed = random_seed, with_self_loops=with_self_loops).astype(np.float64)
        lam_Ga, _ = largest_eigen(A_Ga, max_iter, tol)

        print(f"Sampled A_Ga spectral radius {lam_Ga}")

        katz_cent_Ga = calculate_katz(A_Ga, alpha = alpha, beta=beta, num_iters=max_iter, tol=tol)

        katz_l2_diff = np.linalg.norm(katz_cent_Ga - katz_cent)
        katz_l2_diff_bound_neumann = (alpha * (lam_G + lam_Ga) * clusterLoss) / (1 - alpha * lam_Ga)

        results['katz_l2_diff'] = katz_l2_diff
        results['katz_l2_diff_bound_neumann'] = katz_l2_diff_bound_neumann
        results['Clustering Loss'] = clusterLoss

        data_dict[k] = results


    return {"data_dict": data_dict, "original_nest":{}, "katz_cent_norm":katz_cent_norm}


import plotly.graph_objects as go

def plot_convergence_data(data, title, plot_nest=True):
    data_dict = data["data_dict"]

    # Create traces for each metric
    traces = []
    traces.append(go.Scatter(x=list(data_dict.keys()),
                                    y=[data_dict[k]['Clustering Loss'] for k in data_dict.keys()],
                                    mode='lines',#+markers',
                                    name=r'$\ell$'))
    # Add a constant line at y = 5
    traces.append(go.Scatter(x=list(data_dict.keys()),
                            y=[data["katz_cent_norm"],]*len(data_dict),  # This creates a list of 5's of the same length as the data_dict keys
                            mode='lines',
                            name='Norm of Katz Centrality for G'))
    traces.append(go.Scatter(x=list(data_dict.keys()),
                                    y=[data_dict[k]['katz_l2_diff'] for k in data_dict.keys()],
                                    mode='lines',#+markers',
                                    name=r'Katz L2 Norm Diff (Directed Katz Variation NeSt)'))
    traces.append(go.Scatter(x=list(data_dict.keys()),
                                    y=[data_dict[k]['katz_l2_diff_bound_neumann'] for k in data_dict.keys()],
                                    mode='lines',#+markers',
                                    name=r'Katz L2 Norm Diff Bound (Neumann)'))

    layout_loglog = go.Layout(title=title,
                            xaxis_title='k',
                            yaxis_title='Metric Values (Log Scale)',
                            yaxis_type='log',  # Set y-axis to log scale
                            # Add a second y-axis to the layout
                            yaxis=dict(
                                exponentformat='e',  # Use scientific notation for the primary y-axis
                            ),
                            yaxis2=dict(title='Color Subgraph Count Metrics',
                                        titlefont=dict(color='rgba(148, 103, 189, 1)'),
                                        tickfont=dict(color='rgba(148, 103, 189, 1)'),
                                        overlaying='y',  # This places the second y-axis on top of the first
                                        side='right',  # This places the second y-axis on the right
                                        type='linear'),  # Set the second y-axis to linear scale
                            hovermode='closest',
                            height=900,
                            legend=dict(
                                    orientation="h",
                                    x=0.5,
                                    y=-0.1,
                                    xanchor="center",
                                    yanchor="top"
                                )
                            )




    # Add the new trace to the figure
    fig = go.Figure(data=traces, layout=layout_loglog)
    fig.layout.template = 'simple_white+gridon' # 'presentation'

    fig.show()


# Erdos Renyis

In [5]:
# Get a Graph
n = 400
p = 20/400

G = nx.erdos_renyi_graph(n, p, directed=True)

print(G, f"number of selfloops {nx.number_of_selfloops(G)}")

data_dict = get_convergence_data_of_nest(G, min_cluster_size = 1, alpha=0.04, beta=1, max_k=400, with_self_loops=False)
plot_convergence_data(data_dict, f"Directed Erdos Renyi Graph: n={n}, p={p}")

Spectral Radius: 8.12544892208031
max_degree: 13
Katz converged after 145 iterations.
1: 
Sampled A_Ga spectral radius 7.251552988969716
Katz converged after 94 iterations.
2: 
Sampled A_Ga spectral radius 7.80095182336664
Katz converged after 122 iterations.
3: 
Sampled A_Ga spectral radius 7.980325475266797
Katz converged after 134 iterations.
4: 
Sampled A_Ga spectral radius 8.00675068753534
Katz converged after 136 iterations.
5: 
Sampled A_Ga spectral radius 8.025866443220503
Katz converged after 137 iterations.
6: 
Sampled A_Ga spectral radius 8.060589146513994
Katz converged after 140 iterations.
7: 
Sampled A_Ga spectral radius 8.088769723236103
Katz converged after 142 iterations.
8: 
Sampled A_Ga spectral radius 8.087292072339926
Katz converged after 142 iterations.
9: 
Sampled A_Ga spectral radius 8.123828336153636
Katz converged after 145 iterations.
10: 
Sampled A_Ga spectral radius 8.108683479433807
Katz converged after 144 iterations.
11: 
Sampled A_Ga spectral radius 8.

In [6]:
p = 0.1
n = 150
G = nx.erdos_renyi_graph(n, p)
data_dict = get_convergence_data_of_nest(G, min_cluster_size = 1, max_k = 150)
plot_convergence_data(data_dict, f"Erdős-Rényi: n={n}, p={p}")

Spectral Radius: 15.535395181176424
max_degree: 24


AssertionError: for katz to converge alpha needs to be smaller than 1/lam_G (0.06436913823805765)

In [ ]:
p = 0.25
n = 150
G = nx.erdos_renyi_graph(n, p)
data_dict = get_convergence_data_of_nest(G, min_cluster_size = 1, max_k = 150)
plot_convergence_data(data_dict, f"Erdős-Rényi: n={n}, p={p}")

In [ ]:
p = 0.5
n = 150
G = nx.erdos_renyi_graph(n, p)
data_dict = get_convergence_data_of_nest(G, min_cluster_size = 1, max_k = 150)
plot_convergence_data(data_dict, f"Erdős-Rényi: n={n}, p={p}")

In [ ]:
p = 0.95
n = 150
G = nx.erdos_renyi_graph(n, p)
data_dict = get_convergence_data_of_nest(G, min_cluster_size = 1, max_k = 150)
plot_convergence_data(data_dict, f"Erdős-Rényi: n={n}, p={p}")

# Worst Case: Path Graphs
- almost certain that the cases where the bound is violated are numeric errors from clustering/power iteration, can't easily increase resolution either as optimalclustering works on floats no matter what

In [ ]:
n = 30  # Number of nodes for both graphs
G = nx.path_graph(n)

print("G edges", G.number_of_edges())

data_dict = get_convergence_data_of_nest(G, min_cluster_size = 1)
plot_convergence_data(data_dict, f"Path Graph: Length={n}")

In [ ]:
n = 101  # Number of nodes for both graphs
G = nx.path_graph(n)

print("G edges", G.number_of_edges())

data_dict = get_convergence_data_of_nest(G, min_cluster_size = 1)
plot_convergence_data(data_dict, f"Path Graph: Length={n}")

In [ ]:
n = 200  # Number of nodes for both graphs
expected_edges = 3
total_possible_edges = n * (n - 1) / 2
p = expected_edges / total_possible_edges  # Probability for Erdos-Renyi graph

# Generating the path and Erdos-Renyi graphs with specified parameters
seed = 3
path_graph_redefined = nx.path_graph(n)
G_erdos_renyi_redefined = nx.gnp_random_graph(n, p, seed=seed)
G = nx.compose(path_graph_redefined, G_erdos_renyi_redefined)

print("G edges", G.number_of_edges())

data_dict = get_convergence_data_of_nest(G, min_cluster_size = 1)
plot_convergence_data(data_dict, f"Path + Random Edges: n={n}, #RandomEdges={G.number_of_edges() - path_graph_redefined.number_of_edges()}")

In [ ]:
n = 200  # Number of nodes for both graphs
expected_edges = 3
total_possible_edges = n * (n - 1) / 2
p = expected_edges / total_possible_edges  # Probability for Erdos-Renyi graph

# Generating the path and Erdos-Renyi graphs with specified parameters
path_graph_redefined = nx.path_graph(n)
G_erdos_renyi_redefined = nx.gnp_random_graph(n, p, seed=1)
G = nx.compose(path_graph_redefined, G_erdos_renyi_redefined)

print("G edges", G.number_of_edges())

data_dict = get_convergence_data_of_nest(G, min_cluster_size = 1)
plot_convergence_data(data_dict, f"Path + Random Edges: n={n}, #RandomEdges={G.number_of_edges() - path_graph_redefined.number_of_edges()}")

In [ ]:
n = 201
G = nx.path_graph(n)
G.add_edge(50, 150)
G.add_edge(95, 105)

data_dict = get_convergence_data_of_nest(G, min_cluster_size = 1, max_k = 200, random_seed=333, tol=1e-7)
plot_convergence_data(data_dict, f"Bound Failure: Dominant Eigenvector Switching: Path + Random Edges: n={n}, #RandomEdges={2}")

# Tree Graphs (also difficult)

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

r = 4
d = 5
n = r**d - 1
G = nx.full_rary_tree(r, n)

data_dict = get_convergence_data_of_nest(G, min_cluster_size = 1, max_k = 100)
plot_convergence_data(data_dict, f"r-ary Tree: r={r}, Depth={d}")

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

r = 2
d = 8
n = r**d - 1
G = nx.full_rary_tree(r, n)

data_dict = get_convergence_data_of_nest(G, min_cluster_size = 1, max_k = 100)
plot_convergence_data(data_dict, f"r-ary Tree: r={r}, Depth={d}")

In [ ]:
r = 2
d = 8
n = r**d - 1
print(n)
avg_degree = r+1 # tree has degree = r+1
p_sparse = avg_degree/n*0.01
print("Expected noisy edges added:", n**2*p_sparse)
G = nx.compose(nx.erdos_renyi_graph(n, p_sparse), nx.full_rary_tree(r, n))

data_dict = get_convergence_data_of_nest(G, min_cluster_size = 1, max_k = 300)
plot_convergence_data(data_dict, f"Noisy r-ary Tree: r={r}, Depth={d}, Noise={p_sparse*100:.5f}%")

# d regular Graphs
- WL and EV is constant for exact d regular graphs. d regular graphs are however great expanders i.e. have a big spectral gap so bound should be good here.
- need to slightly pertub them

In [ ]:
p = 109
expected_edges = 10
total_possible_edges = p * (p - 1) / 2
p_sparse = expected_edges / total_possible_edges  # Probability for Erdos-Renyi graph

Gpaley = nx.paley_graph(p).to_undirected()
G = nx.compose(nx.erdos_renyi_graph(p, p_sparse, seed=57), Gpaley)
print(f"number of edges in Gpaley {Gpaley.number_of_edges()} number of edges added by noise {G.number_of_edges() - Gpaley.number_of_edges()}")

data_dict = get_convergence_data_of_nest(G, min_cluster_size = 1, max_k = 100)
plot_convergence_data(data_dict, f"Paley Graph + Noise: p={p}, NoiseLevel={p_sparse*100:.4f}%")

In [ ]:
n = 400
d = 3
G = nx.random_regular_graph(d, n)
data_dict = get_convergence_data_of_nest(G, min_cluster_size = 1)
plot_convergence_data(data_dict, "d-Regular Graphs have a constant perron eigenvector still, this shows the upper bound works even here")